In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import urllib
import numpy as np

In [2]:
response = requests.get("http://results.enr.clarityelections.com/GA/63991/184321/en/select-county.html")
content = response.content

In [3]:
response.status_code

200

In [4]:
soup = BeautifulSoup(content, 'html.parser')

In [5]:
county_table = soup.find_all('table')[6]


In [6]:
# get all counties and links in a dataframe

rows = []
# loop through all links and return the name and link of each county
for a in county_table.find_all('a'):
    # enter county and value into a dictionary
    row = {}
    county = a.get('id')
    link = a.get('value')
    county_id = link.split('/')[2]
    row.update({"county": county,"county_id":county_id}) 

    #append dictionary to list
    rows.append(row)

# enter list into dataframe
df = pd.DataFrame(rows) 

df

,county,county_id
0,Appling,63993
1,Atkinson,63994
2,Bacon,63995
3,Baker,63996
4,Baldwin,63997
5,Banks,63998
6,Barrow,63999
7,Bartow,64000
8,Ben Hill,64001
9,Berrien,64002


In [7]:
results = []

def get_candidate(name):
    if "trump" in name.lower(): 
        return_name = name.replace("(REP)","").replace("(LIB)","").strip().title()
    elif "clinton" in name.lower():
        return_name = name.replace("(DEM)","").replace("(LIB)","").strip().title()
    else:
        return_name = "Other"
    return return_name
        

# loop through links for each county and return results for each candidate
for i in range(len(df)):
    county = df.county.loc[i]
    link = 'https://results.enr.clarityelections.com/GA/'+county.replace(" ", "_")+'/'+df.county_id.loc[i]+'/current_ver.txt'
    response = requests.get(link)
    content = response.content
    version = content.decode("utf-8")
    file = 'http://results.enr.clarityelections.com/GA/'+county.replace(" ", "_")+'/'+df.county_id.loc[i]+'/'+version+'/json/sum.json'
#     print(link)
    r = urllib.request.urlopen(file)
    data = json.loads(r.read().decode(r.info().get_param('charset') or 'utf-8'))
    values = data['Contests'][0]['V']
    candidates = data['Contests'][0]['CH']
#     print(values,candidates)
    for j in range(len(values)):
        row = {}
        candidate = get_candidate(candidates[j])
#         candidate = candidates[j].replace("(REP)","").replace("(DEM)","").replace("(LIB)","").strip()
        votes = values[j]
        row.update({"county":county,"candidate":candidate,"votes":votes})
        
        results.append(row)
        
# enter list into dataframe
georgia = pd.DataFrame(results) 

georgia

,candidate,county,votes
0,Donald J. Trump,Appling,5494
1,Hillary Clinton,Appling,1434
2,Other,Appling,77
3,Donald J. Trump,Atkinson,1878
4,Hillary Clinton,Atkinson,697
5,Other,Atkinson,35
6,Donald J. Trump,Bacon,3364
7,Hillary Clinton,Bacon,608
8,Other,Bacon,47
9,Donald J. Trump,Baker,775


In [8]:
georgia.to_csv('georgia.csv')